# **SharePoint and Dataverse/Dynamics REST API for Data Validation**

This notebook demonstrates how to connect to SharePoint and Dataverse using the REST API in Python. The use case was data validation after a data migration. After authenticating, all data is pulled from both SharePoint and Dataverse, and then compared row by row to find any differences. Recursively pulls 5000 results per request. Successfully used with tables containing over 1M rows.

In [ ]:
import requests
import json
import pickle 
import time
import pandas as pd

In [ ]:
# pandas display options 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## **SHAREPOINT**

In [ ]:
SHAREPOINT_URL = "https://XXXXXXXXXXXXXX.sharepoint.com/sites/XXXXXXXXXXXXXXXXX"

### REST API using rtFa and FedAuth Cookies

In [ ]:
# if unable to use app  principal, you can get your cookies directly in the sharepoint web interface
# browser F12 developer tools -> networking -> open a sharepoint list -> look for something like getallitems call, copy cookies header
SHAREPOINT_COOKIES = "rtFa=XXXXXXXXXXXXXXXXXX; FedAuth=XXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
# sharepoint table
sharepoint_table_name = "YOUR_TABLE_NAME"
sharepoint_columns = [
    "YOUR_COLUMN_1",
    "YOUR_COLUMN_2",
    "YOUR_COLUMN_3", 
]

In [ ]:
# request headers  
sharepoint_session = requests.Session()
sharepoint_session.headers.update(
    {
        "authority" : "XXXXXXXX.sharepoint.com",
        'accept':'application/json;odata=nometadata',
        "accept-encoding" : "gzip, deflate, br",
        'content-type': 'application/json;charset=utf-8;odata=nometadata',
        "cookie" : SHAREPOINT_COOKIES,
        "user-agent" : f"python-requests/{requests.__version__}",
        "connection" : "keep-alive",
    }
)

In [ ]:
# prep request url
sharepoint_columns_comma_seperated = ",".join(sharepoint_columns)
sharepoint_select_columns_and_page_size = f"?$select={sharepoint_columns_comma_seperated}&$top=5000"
sharepoint_request_uri = f"{SHAREPOINT_URL}/_api/web/lists/getbytitle('{sharepoint_table_name}')/items"
sharepoint_request_uri += sharepoint_select_columns_and_page_size
print(sharepoint_request_uri)

In [ ]:
# request loop
sharepoint_start_time = time.time()
try:
    sharepoint_result = sharepoint_session.get(sharepoint_request_uri)
    sharepoint_result_json = json.loads(sharepoint_result.content)
except:
    raise Exception(f"{sharepoint_result}, {sharepoint_result.content}")
sharepoint_result_json_value = sharepoint_result_json["value"]
print("Items Read: ", len(sharepoint_result_json_value), sep="", end="")
while "odata.nextLink" in sharepoint_result_json:
    sharepoint_result = sharepoint_session.get(sharepoint_result_json["odata.nextLink"])
    sharepoint_result_json = json.loads(sharepoint_result.content)
    sharepoint_result_json_value += sharepoint_result_json["value"]
    print("...", len(sharepoint_result_json_value), sep="", end="")
sharepoint_end_time = time.time()
print("\nRuntime:", round((sharepoint_end_time-sharepoint_start_time)/60, 2), "min")

In [ ]:
# convert to pandas dataframe 
df_sharepoint_raw = pd.DataFrame(sharepoint_result_json_value, dtype="object")[sharepoint_columns]
df_sharepoint_raw.info()

### Save/Load Raw Data

In [ ]:
# save file
with open("df_sharepoint_raw.pkl", "wb") as f1:
    pickle.dump(df_sharepoint_raw, f1)

In [ ]:
# load file
with open("df_sharepoint_raw.pkl", "rb") as f1:
    df_sharepoint_raw = pickle.load(f1)

### Alternative Auth Method #1: Service Principal with REST API

In [ ]:
# reference: https://martinnoah.com/sharepoint-rest-api-with-python.html
# get access token
client_id = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
client_secret = "XXXXXXXXXXXXXXXXXXXXXX"
tenant =  'XXXXXXXXXXXX'
tenant_id = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'  
client_id = client_id + '@' + tenant_id
data = {
    'grant_type':'client_credentials',
    'resource': "00000003-0000-0ff1-ce00-000000000000/" + tenant + ".sharepoint.com@" + tenant_id, 
    'client_id': client_id,
    'client_secret': client_secret,
}
headers = {
    'Content-Type':'application/x-www-form-urlencoded'
}
url = f"https://accounts.accesscontrol.windows.net/{tenant_id}/tokens/OAuth/2"
r = requests.post(url, data=data, headers=headers)
json_data = json.loads(r.text)
print(json_data)

In [ ]:
# sharepoint rest api request example
sharepoint_site_name = "XXXXXXXXXXXXX"
headers = {
    'Authorization': "Bearer " + json_data['access_token'],
    'Accept':'application/json;odata=verbose',
    'Content-Type': 'application/json;odata=verbose'
}
url = f"https://{tenant}.sharepoint.com/sites/{sharepoint_site_name}/_api/web/lists/getbytitle('{sharepoint_table_name}')/items"
requests.get(url, headers=headers)

### Alternative Auth Method #2: Service Principal with Office365-Sharepoint API 

In [ ]:
from office365.sharepoint.client_context import ClientContext, ClientCredential

In [ ]:
# sharepoint auth params 
CLIENT_ID = "XXXXXXXXXXXXXXXXXXXXXXX"
CLIENT_SECRET = "XXXXXXXXXXXXXXXXXXXXXX"
SHAREPOINT_URL = "https://XXXXXXXXXXXXXXXXX.sharepoint.com/sites/XXXXXXXXXXXXXXXXXXX"

In [ ]:
# sharepoint app service principal auth
# how to setup: https://github.com/vgrem/Office365-REST-Python-Client/wiki/How-to-connect-to-SharePoint-Online-and-and-SharePoint-2013-2016-2019-on-premises--with-app-principal
ctx = ClientContext(SHAREPOINT_URL).with_client_credentials(CLIENT_ID, CLIENT_SECRET)
target_web = ctx.web.get().execute_query()
print(target_web.url)

In [ ]:
# download sharepoint data 
sharepoint_site_pages = ctx.web.lists.get_by_title(sharepoint_table_name)
sharepoint_all_items = sharepoint_site_pages.items.get_all(
    5_000, lambda x:print("Items Read:", len(x))
).select(sharepoint_columns).execute_query()

In [ ]:
# convert to pandas dataframe 
sharepoint_all_items_list = []
for each_item in sharepoint_all_items:
    sharepoint_all_items_list.append(each_item.properties)
df_sharepoint_raw = pd.DataFrame(sharepoint_all_items_list, dtype="object")

## **DATAVERSE**

In [ ]:
DATAVERSE_URL = "https://XXXXXXXXXXXXX.XXXXXX.dynamics.com"

### REST API using Access Token

In [ ]:
# if unable to use interactive login below, you can get your token directly in the dataverse web interface
# F12 browser developer tools -> networking -> open a dv table -> look for something like a getclientmetadata call, copy bearer token
DATAVERSE_ACCESS_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
# dataverse table info
dataverse_table_name = "YOUR_TALBE_NAME"
dataverse_columns = [
    "YOUR_COLUMN_1",
    "YOUR_COLUMN_2",
    "YOUR_COLUMN_3", 
]

In [ ]:
# request session headers 
dataverse_session = requests.Session()
dataverse_session.headers.update(
    {
        "Authorization" : f"Bearer {DATAVERSE_ACCESS_TOKEN}",
        'OData-MaxVersion': '4.0', 
        'OData-Version': '4.0', 
        'If-None-Match': 'null', 
        'Accept': 'application/json',
        "Content-Type": "application/json; charset=utf-8",
        "Prefer" : "odata.maxpagesize=5000",
        "User-Agent" : f"python-requests/{requests.__version__}",
        "Connection" : "keep-alive",
    }
)

In [ ]:
# prep request 
dataverse_columns_comma_seperated = ",".join(dataverse_columns)
dataverse_select_columns = f"?$select={dataverse_columns_comma_seperated}"
dataverse_request_uri = f'{DATAVERSE_URL}/api/data/v9.2/{dataverse_table_name}'
dataverse_request_uri += dataverse_select_columns
print(dataverse_request_uri)

In [ ]:
# request loop
dataverse_start_time = time.time()
try:
    dataverse_result = dataverse_session.get(dataverse_request_uri)
    dataverse_result_json = json.loads(dataverse_result.content)
except:
    raise Exception(f"{dataverse_result}, {dataverse_result.content}")
dataverse_result_json_value = dataverse_result_json["value"]
print("Items Read: ", len(dataverse_result_json_value), sep="", end="")
while "@odata.nextLink" in dataverse_result_json:
    dataverse_result = dataverse_session.get(dataverse_result_json["@odata.nextLink"])
    dataverse_result_json = json.loads(dataverse_result.content)
    dataverse_result_json_value += dataverse_result_json["value"]
    print("...", len(dataverse_result_json_value), sep="", end="")
dataverse_end_time = time.time()
print("\nRuntime:", round((dataverse_end_time-dataverse_start_time)/60, 2), "min")

In [ ]:
# convert to pandas dataframe 
df_dataverse_raw = pd.DataFrame(dataverse_result_json_value, dtype="object")[dataverse_columns]
df_dataverse_raw.info()

### Save/Load Raw Data

In [ ]:
# save files
with open("df_dataverse_raw.pkl", "wb") as f2:
    pickle.dump(df_dataverse_raw, f2)

In [ ]:
# load files
with open("df_dataverse_raw.pkl", "rb") as f2:
    df_dataverse_raw = pickle.load(f2)

### Alternative Auth Method: MSAL Auth via Interactive Web Browser Log-In

In [ ]:
# how to setup: https://andyhowes.co/easily-connect-to-dataverse-microsoft-dynamics-with-python/
CLIENT_ID = "XXXXXXXXXXXXXXXXXXXXXX"
TENANT_ID = "XXXXXXXXXXXXXXXXXXXXXXXXXX"
AUTHORITY_BASE = "https://login.microsoftonline.com"
SCOPE_SUFFIX = "user_impersonation"

In [ ]:
# interactive browser auth, note: app registration must have localhost redirect 
import msal
AUTHORITY_URL = f"{AUTHORITY_BASE}/{TENANT_ID}"
SCOPE = f"{DATAVERSE_URL}/{SCOPE_SUFFIX}"
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY_URL)
auth_result = app.acquire_token_interactive([SCOPE])
DATAVERSE_ACCESS_TOKEN = auth_result["access_token"]

## Data Processing

In [ ]:
# copies for data processing 
df_sharepoint_processed = df_sharepoint_raw.copy()
df_dataverse_processed = df_dataverse_raw.copy()

In [ ]:
# column names
df_dataverse_processed = df_dataverse_processed.rename(
    columns={
        'BEFORE_1' : 'AFTER_1',
        'BEFORE_2' : 'AFTER_2',
        'BEFORE_3' : 'AFTER_3',
    }
)

In [ ]:
# column order
column_order = [
    'YOUR_COLUMN_1',
    'YOUR_COLUMN_2',
    'YOUR_COLUMN_3'
]
df_sharepoint_processed = df_sharepoint_processed[column_order]
df_dataverse_processed= df_dataverse_processed[column_order]

In [ ]:
# empty values
df_sharepoint_processed = df_sharepoint_processed.fillna("Null")
df_dataverse_processed = df_dataverse_processed.fillna("Null")
df_sharepoint_processed = df_sharepoint_processed.replace("", "Null")
df_dataverse_processed = df_dataverse_processed.replace("", "Null")

In [ ]:
# trim whitespace values 
df_sharepoint_processed = df_sharepoint_processed.map(lambda x: x.strip() if isinstance(x, str) else x)
df_dataverse_processed = df_dataverse_processed.map(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
# dataverse choice list
convert = {
    "BEFORE_1" : "AFTER_1",
    "BEFORE_2" : "AFTER_2",
    "BEFORE_3" : "AFTER_3"
}
df_dataverse_processed["YOUR_CHOICE_LIST_COLUMN"] = df_dataverse_processed["YOUR_CHOICE_LIST_COLUMN"].apply(
    lambda x:convert[str(x)]
)

In [ ]:
# shapes 
print("sharepoint:", df_sharepoint_processed.shape)
print("dataverse:", df_dataverse_processed.shape)

## Data Compairison

In [ ]:
# fix type errors 
df_sharepoint_processed["YOUR_COLUMN"] = df_sharepoint_processed["YOUR_COLUMN"].astype("str")
df_dataverse_processed["YOUR_COLUMN"] = df_dataverse_processed["YOUR_COLUMN"].astype("str")

In [ ]:
# outer join
outer_join = df_sharepoint_processed.merge(
    df_dataverse_processed, 
    on=column_order,
    how='outer', 
    indicator=True
)

In [ ]:
# anti-join
anti_join_sharepoint = outer_join[(outer_join._merge=='left_only')].drop('_merge', axis=1)
anti_join_dataverse = outer_join[(outer_join._merge=='right_only')].drop('_merge', axis=1)
anti_join_sharepoint["SourceTable"] = "Sharepoint"
anti_join_dataverse["SourceTable"] = "Dataverse"
anti_join = pd.concat([anti_join_sharepoint, anti_join_dataverse])

In [ ]:
# sort results
anti_join = anti_join.sort_values(
    by=["YOUR_COLUMN_1","YOUR_COLUMN_2","YOUR_COLUMN_3"]
).reset_index(drop=True)

In [ ]:
# size of results 
print("sharepoint:", df_sharepoint_processed.shape[0])
print("dataverse:", df_dataverse_processed.shape[0])
print("sharepoint mismatches:", anti_join_sharepoint.shape[0])
print("dataverse mismatches:", anti_join_dataverse.shape[0])
print("mismatch sum:", anti_join.shape[0])

## Exploratory Data Analysis 

In [ ]:
anti_join[0:50]

In [ ]:
def check_series(s1, s2):
    """
    Function that checks 2 rows and displays the columns/values that are different
    """
    mask = s1 == s2
    if sum(~mask.values) <= 1:
        print("Rows are Equal")
        return None
    else:
        s1_source = s1.SourceTable 
        s2_source = s2.SourceTable
        s1_mismatch = s1[~mask]
        s2_mismatch = s2[~mask]
        df = pd.concat([s1_mismatch.drop("SourceTable"), s2_mismatch.drop("SourceTable")], axis=1)
        df.columns = [s1_source, s2_source]
        print( "Rows are NOT equal")
        return df

In [ ]:
check_series(anti_join.iloc[0], anti_join.iloc[0])

In [ ]:
check_series(anti_join.iloc[0],anti_join.iloc[1])

In [ ]:
check_series(anti_join.iloc[2],anti_join.iloc[3])

In [ ]:
check_series(anti_join.iloc[0],anti_join.iloc[3])